In [1]:
import re
from faker import Faker
import spacy
import random
from spacy.tokens import DocBin

fake = Faker()

In [31]:
templates = [
    "Hospital Name: Sunshine Clinic, Patient ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Gender - {PATIENT-GENDER}",
    "Hospital:  Moonlight Hospital, Patient ID:  {PATIENT-ID}, Name:  {PATIENT-NAME}, Gender: {PATIENT-GENDER}, Address: {PATIENT-ADDRESS}, Date of Birth: {PATIENT-DOB}",
    "Starlight Medical Center, Patient ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Gender - {PATIENT-GENDER}, Address - {PATIENT-ADDRESS}, Birth - {PATIENT-DOB}, Admit Date - {PATIENT-ADMIT-DATE}, Discharge Date - {PATIENT-DISCHARGE-DATE}",
    "Hospital Name: Sunset General Hospital, ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Address - {PATIENT-ADDRESS}, Type - {PATIENT-TYPE}, DOB - {PATIENT-DOB}, Gender - {PATIENT-GENDER}, Admit Date - {PATIENT-ADMIT-DATE}, Discharge Date - {PATIENT-DISCHARGE-DATE}, DocID - {PATIENT-DOCID}",
    "Hospital : Sunrise Medical Center, Patient ID - {PATIENT-ID}, Name - {PATIENT-NAME}",
    "Hospital Name: Twilight Hospital, Patient ID:- {PATIENT-ID}, Name:- {PATIENT-NAME}, Gender:- {PATIENT-GENDER}, Address:- {PATIENT-ADDRESS}",
    "Starlight Medical Center, Patient ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Gender - {PATIENT-GENDER}, Birth - {PATIENT-DOB}, Admit Date - {PATIENT-ADMIT-DATE}",
    "Hospital Name: Dawn General Hospital, ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Gender - {PATIENT-GENDER}, Address - {PATIENT-ADDRESS}, DOB - {PATIENT-DOB}, DocID - {PATIENT-DOCID}",
    "Name: Nightfall Hospital, ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Address - {PATIENT-ADDRESS}, Type - {PATIENT-TYPE}, DOB - {PATIENT-DOB}",
    "Evening Medical Center, Patient ID - {PATIENT-ID}, Gender - {PATIENT-GENDER}, DOB - {PATIENT-DOB}, Admit Date - {PATIENT-ADMIT-DATE}, Discharge Date - {PATIENT-DISCHARGE-DATE}",
    "Hospital Name: Morning General Hospital, ID - {PATIENT-ID}, Gender - {PATIENT-GENDER}, Admit Date - {PATIENT-ADMIT-DATE}, Discharge Date - {PATIENT-DISCHARGE-DATE}, DocID - {PATIENT-DOCID}",
    "Hospital Name: Afternoon Clinic, Name - {PATIENT-NAME}, Gender - {PATIENT-GENDER}, Admit Date - {PATIENT-ADMIT-DATE}, Discharge Date - {PATIENT-DISCHARGE-DATE}",
    "Hospital : Daybreak Medical Center, Patient ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Address - {PATIENT-ADDRESS}, Date of Birth - {PATIENT-DOB}, Gender - {PATIENT-GENDER}",
    "Hospital Name: Midnight Hospital, ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Birth - {PATIENT-DOB}, Admit Date - {PATIENT-ADMIT-DATE}, Discharge Date - {PATIENT-DISCHARGE-DATE}",
    "Name: Midday General Hospital, Patient ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Gender - {PATIENT-GENDER}, Address - {PATIENT-ADDRESS}, DOB - {PATIENT-DOB}, DocID - {PATIENT-DOCID}"
    """Community General Hospital Patient Account# {PATIENT-ID} Patient Name {PATIENT-NAME} Address {PATIENT-ADDRESS} Previous or Maiden \
    Name Notify in Case of Emergency Patient Social Security Number Employer Name Big Company Guarantor Address Same Payer Financial Class Provisional Diagnosis \
    Right Ing Hernia Principal Diagnosis, Secondary Diagnosis and Complications INPATIENT REGISTRATION AND SUMMARY FORM Medical Record# {PATIENT-DOCID} Admit \
    Date Admit time DateofPrev. Admit Attending Physician Number and Name Birth) Date Patient Type {PATIENT-TYPE} Marital Religion St Hospital Services \
    S {PATIENT-ADMIT-DATE} 11:10 Patient Phone# 123-123-4567 Comments Donor No Phone Guarantor# # Guarantor Name 27 Cottonwood Ln Anytown USA Age {PATIENT-DOB} \
    54 {PATIENT-GENDER} M Address City/State Relationship Employer City/State Anytown, US John Brown Guarantor Social Security Number Group Name Dischargel Date \
    Disch. Time {PATIENT-DISCHARGE-DATE} /11:30 Policy Number 174726236514 Insured's Name John Brown Southern Company Admitted By AD12 Patient Weight 550, 91 \
    3144 Principal Procedures and Secondary Procedures 53.03 633 49530 1-12 More Consultations Disposition ofCase < Home Expired Transferred to: Coder kp Date \
    Swing Bed Autopsy Supervised Living AMA D Home Health Nursing Home I certify that the narrative description of the principal and secondary diagnoses and the \
    major procedures performed are accurate and complete to the best ofr my knowledge. Attending Physician Date""",
    "{PATIENT-NAME} (ID: {PATIENT-ID}), {PATIENT-GENDER}, DOB: {PATIENT-DOB}, Addr: {PATIENT-ADDRESS} (Optional) | Key: Admit: {PATIENT-ADMIT-DATE} (Optional), Discharge: {PATIENT-DISCHARGE-DATE} (Optional), DocID: {PATIENT-DOCID} (Optional), Type: {PATIENT-TYPE} (Optional)",
    "Name: {PATIENT-NAME}? ID: {PATIENT-ID}? Date of Birth: {PATIENT-DOB}? Gender: {PATIENT-GENDER}? Address: {PATIENT-ADDRESS}? (Optional)",
    "Hosp: Ruby Hall, ID: {PATIENT-ID}, Name: {PATIENT-NAME}, Gender: {PATIENT-GENDER}",
    "Welcome {PATIENT-NAME} (ID: {PATIENT-ID}), born {PATIENT-DOB}. (Optional: We care for {PATIENT-TYPE} s.)",
    "Arrived: {PATIENT-ADMIT-DATE}. Patient: {PATIENT-NAME} ({PATIENT-ID}), {PATIENT-GENDER}, DOB: {PATIENT-DOB}, Addr: {PATIENT-ADDRESS} (Optional). Treatment (Optional): Brief summary. Discharged: {PATIENT-DISCHARGE-DATE} (Optional)",
    "{PATIENT-NAME} ({PATIENT-ID}), {PATIENT-GENDER}, Birth: {PATIENT-DOB}, Addr: {PATIENT-ADDRESS} (Optional) | Key Dates: Admit: {PATIENT-ADMIT-DATE} (Optional), Discharge: {PATIENT-DISCHARGE-DATE} (Optional)",
    "{PATIENT-ID} | {PATIENT-NAME} | {PATIENT-GENDER} | {PATIENT-DOB} | {PATIENT-ADDRESS} (Optional)",
    "{PATIENT-NAME} ({PATIENT-ID}), {PATIENT-GENDER}, Birth: {PATIENT-DOB}. Next: Review details, Meet doctor.",
    "{PATIENT-NAME} {PATIENT-GENDER} (ID: {PATIENT-ID}), DOB: {PATIENT-DOB} , Addr: {PATIENT-ADDRESS} (Optional) | Info (Optional): Admit: {PATIENT-ADMIT-DATE}️, Discharge: {PATIENT-DISCHARGE-DATE}️ (Optional), DocID: {PATIENT-DOCID}",
    "{PATIENT-NAME} (ID: {PATIENT-ID}), DOB: {PATIENT-DOB}, Gender: {PATIENT-GENDER}, Address: {PATIENT-ADDRESS} (Optional)",
    "Patient Name: {PATIENT-NAME} ID: {PATIENT-ID} Birth: {PATIENT-DOB} Gender: {PATIENT-GENDER} Address: {PATIENT-ADDRESS} (Optional)",
    "{PATIENT-NAME} ({PATIENT-GENDER})ID: {PATIENT-ID} DOB: {PATIENT-DOB} Address: {PATIENT-ADDRESS} (Optional) Admitted: {PATIENT-ADMIT-DATE} (Optional)",
    "Patient Registration Name: {PATIENT-NAME} ID: {PATIENT-ID} Gender: {PATIENT-GENDER} Date of Birth: {PATIENT-DOB} Address: {PATIENT-ADDRESS} (Optional) Doctor: {PATIENT-DOCID} (Optional)",
    """Welcome {PATIENT-NAME}! ID: {PATIENT-ID} Gender: {PATIENT-GENDER} Birth: {PATIENT-DOB} Address: {PATIENT-ADDRESS} (Optional) Admit Date: {PATIENT-ADMIT-DATE} (Optional)""",
]

In [185]:
# # Placeholder to Faker generator mapping
# placeholder_to_generator = {
#     "PATIENT-ID": lambda: fake.random_element(
#         elements=(
#             fake.bothify(text="???######"),
#             fake.bothify(text="P-???#####"),
#             fake.bothify(text="########"),
#         )
#     ),
#     "PATIENT-NAME": lambda: fake.name(),
#     # "PATIENT-ADDRESS": lambda: fake.address().replace("\n", " "),
#     "PATIENT-ADDRESS": lambda: fake.random_element(elements=("Pune", "Agra")),
#     "PATIENT-DOCID": lambda: fake.random_element(
#         elements=(
#             fake.bothify(text="####"),
#             fake.bothify(text="D-###-####"),
#             fake.bothify(text="DOC-###-####"),
#             fake.bothify(text="######"),
#         )
#     ),
#     "PATIENT-TYPE": lambda: fake.random_element(
#         elements=("Inpatient", "Outpatient", "Inp", "Oup")
#     ),
#     "PATIENT-ADMIT-DATE": lambda: fake.date(),
#     "PATIENT-DOB": lambda: fake.date_of_birth().strftime("%Y-%m-%d"),
#     "PATIENT-GENDER": lambda: fake.random_element(
#         elements=("Male", "Female", "M", "F", "Other", "O")
#     ),
#     "PATIENT-DISCHARGE-DATE": lambda: fake.date(),
# }

In [204]:
# def generate_sample(text, mapping):
#     updated_text = text
#     entities = []
#     offset = 0
#     for placeholder, generator in mapping.items():
#         while True:
#             match = re.search(r"\{" + placeholder + r"\}", updated_text)
#             if not match:
#                 break
#             start, end = match.span()
#             fake_value = generator()
#             updated_text = updated_text[:start] + fake_value + updated_text[end:]
#             new_end = start + len(fake_value)
#             entities.append((start, new_end, placeholder))
#             offset += new_end - end

#     return updated_text, {"entities": entities}


# def generate_sample(text, mapping):
#     updated_text = text
#     entities = []
#     offset = 0

#     # Iterate over each placeholder in the mapping
#     for placeholder, generator in mapping.items():
#         while True:
#             # Find the next occurrence of the placeholder in the updated text
#             match = re.search(r"\{" + placeholder + r"\}", updated_text)
#             if not match:
#                 break
#             start, end = match.span()
#             # Generate the fake value for the placeholder
#             fake_value = generator()
#             updated_text = updated_text[:start] + fake_value + updated_text[end:]
#             new_end = start + len(fake_value)
#             entities.append((start - offset, new_end - offset, placeholder))
#             offset += end - start - len(fake_value)

#     return updated_text, {"entities": entities}


# -----------------------------------------------------------------------------

In [ ]:
# updated_text = "Hospital Name: Sunshine Clinic, Patient ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Gender - {PATIENT-GENDER}"
# mapping = placeholder_to_generator
# sorted_mapping = sorted(
#     mapping.items(), key=lambda x: updated_text.index("{" + x[0] + "}")
# )
# print(sorted_mapping)

In [205]:
# def generate_multiple_samples(n):
#     samples = []
#     for _ in range(n):
#         template_text = random.choice(templates)  # Randomly select a template
#         sample_text, entities = generate_sample(template_text, placeholder_to_generator)
#         samples.append((sample_text, entities))
#     return samples


# def generate_multiple_samples(n):
#     samples = []
#     for _ in range(n):
#         template_text = random.choice(templates)  # Randomly select a template
#         try:
#             sample_text, entities = generate_sample(
#                 template_text, placeholder_to_generator
#             )
#             samples.append((sample_text, entities))
#         except ValueError:
#             pass  # Placeholder not found, skip this template
#     return samples

In [ ]:
# Generate samples
# samples = generate_multiple_samples(2)
# print(samples)
# # Optionally, print or process the samples
# for sample in samples:  # Print only the first sample for brevity
#     print(sample[0])  # Print a portion of the updated text
#     print(sample[1])  # Print the entity information
#     # labels = [label for _, _, label in sample[1]["entities"]]

In [ ]:
# for sample in samples:
#     print(sample)
#     print("\n")

In [ ]:
# Good work on 1
# def verify_samples(samples):
#     verified_samples = []
#     for text, annotation in samples:
#         entities = annotation.get("entities", [])
#         verified_entities = []
#         for start, end, label in entities:
#             entity_text = text[start:end]
#             entity_length = len(entity_text)
#             end = start + entity_length
#             verified_entities.append((entity_text, label, start, end))
#         verified_samples.append((text, verified_entities))
#     return verified_samples


# # Usage
# verified_samples = verify_samples(samples)
# for i, (text, entities) in enumerate(verified_samples):
#     print(f"Sample {i+1}:")
#     print(f"Text: {text}")
#     print("Entities:")
#     for entity_text, label, start, end in entities:
#         print(f"  - Text: {entity_text}, Label: {label}, Start: {start}, End: {end}")
#     print()

In [ ]:
# def verify_sample(text, entities):
#     verified_entities = []
#     for start, end, label in entities:
#         entity_text = text[start:end]
#         entity_length = len(entity_text)
#         end = start + entity_length
#         verified_entities.append((entity_text, label, start, end))
#     return text, verified_entities


# # Sample data
# text = "Hospital Name: Dawn General Hospital, ID - EeO660756, Name - Kimberly Gill, Gender - Other, Address - PSC 7382, Box 8319 APO AP 01050, DOB - 1967-11-27, DocID - DOC-532-7258"
# entities = [
#     (43, 52, "PATIENT-ID"),
#     (61, 74, "PATIENT-NAME"),
#     (113, 144, "PATIENT-ADDRESS"),
#     (175, 187, "PATIENT-DOCID"),
#     (152, 162, "PATIENT-DOB"),
#     (85, 90, "PATIENT-GENDER"),
# ]

# # Verify the sample
# verified_text, verified_entities = verify_sample(text, entities)

# # Print the verified sample
# print("Verified Text:", verified_text)
# print("Entities:")
# for entity_text, label, start, end in verified_entities:
#     print(f"  - Text: {entity_text}, Label: {label}, Start: {start}, End: {end}")

In [ ]:
# text = "Hospital Name: Sunshine Clinic, Patient ID - cCr891258, Name - Tonya Lewis, Gender - Female"
# for i, char in enumerate(text):
#     print(f"Index: {i}, Character: {char}")

TRIAL
Working on 23-01-24 Friday


In [4]:
import re

# THIS IS ONLY FOR MAKING AND CHECK ONE SAMPLE


# Define generator functions for each placeholder
def generate_patient_id():
    return "P-123456"


def generate_patient_name():
    return "John Doe"


def generate_patient_gender():
    return "Male"


# Define a function to replace placeholders in the order of their first occurrence
def replace_placeholders(template):
    updated_text = template
    entities = []
    offset = 0

    # Find all placeholders in the template
    placeholders = re.findall(r"\{([^}]*)\}", template)

    for placeholder in placeholders:
        # Find the start and end positions of the placeholder in the updated text
        start = updated_text.index("{" + placeholder + "}")
        end = start + len("{" + placeholder + "}")

        # Generate the fake value for the placeholder based on its name
        if placeholder == "PATIENT-ID":
            fake_value = generate_patient_id()
        elif placeholder == "PATIENT-NAME":
            fake_value = generate_patient_name()
        elif placeholder == "PATIENT-GENDER":
            fake_value = generate_patient_gender()

        # Replace the placeholder with the fake value in the updated text
        updated_text = updated_text[:start] + fake_value + updated_text[end:]

        # Update the start and end positions of the entities
        entities.append((start, start + len(fake_value), placeholder))
        offset += len(fake_value) - len("{" + placeholder + "}")

    return updated_text, {"entities": entities}


# Example usage
template = "Hospital Name: Sunshine Clinic, Patient ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Gender - {PATIENT-GENDER}"
updated_text, entities = replace_placeholders(template)
print("Updated Text:", updated_text)
print("Entities:", entities)

Updated Text: Hospital Name: Sunshine Clinic, Patient ID - P-123456, Name - John Doe, Gender - Male
Entities: {'entities': [(45, 53, 'PATIENT-ID'), (62, 70, 'PATIENT-NAME'), (81, 85, 'PATIENT-GENDER')]}


In [13]:
from faker import Faker

fake = Faker()


def generate_patient_id():
    return fake.random_element(
        elements=(
            fake.bothify(text="???######"),
            fake.bothify(text="P-???#####"),
            fake.bothify(text="########"),
        )
    )


def generate_patient_name():
    return fake.name()


def generate_patient_address():
    return fake.address().replace("\n", " ")
    # "PATIENT-ADDRESS": lambda: fake.address().replace("\n", " "),


def generate_patient_docid():
    return fake.random_element(
        elements=(
            fake.bothify(text="####"),
            fake.bothify(text="D-###-####"),
            fake.bothify(text="DOC-###-####"),
            fake.bothify(text="######"),
        )
    )


def generate_patient_type():
    return fake.random_element(elements=("Inpatient", "Outpatient", "Inp", "Oup"))


def generate_patient_admit_date():
    # Generate a datetime object
    date_time = fake.date_time_between(start_date="-5y", end_date="now")
    # Format the datetime object as a string
    return date_time.strftime("%m/%d/%Y")


def generate_patient_dob():
    return fake.date_of_birth().strftime("%m/%d/%Y")


def generate_patient_gender():
    return fake.random_element(elements=("Male", "Female", "M", "F", "Other", "O"))


def generate_patient_discharge_date():
    # Generate a datetime object
    date_time = fake.date_time_between(start_date="-5y", end_date="now")
    # Format the datetime object as a string
    return date_time.strftime("%m/%d/%Y")

In [14]:
placeholder_to_generator = {
    "PATIENT-ID": generate_patient_id,
    "PATIENT-NAME": generate_patient_name,
    "PATIENT-ADDRESS": generate_patient_address,
    "PATIENT-DOCID": generate_patient_docid,
    "PATIENT-TYPE": generate_patient_type,
    "PATIENT-ADMIT-DATE": generate_patient_admit_date,
    "PATIENT-DOB": generate_patient_dob,
    "PATIENT-GENDER": generate_patient_gender,
    "PATIENT-DISCHARGE-DATE": generate_patient_discharge_date,
}

In [19]:
def replace_placeholders(template, placeholder_to_generator):
    updated_text = template
    entities = []

    # Iterate over each placeholder in the template text
    while "{" in updated_text:
        # Find the next placeholder in the updated text
        match = re.search(r"\{([^{}]*)\}", updated_text)
        if not match:
            break
        placeholder = match.group(1)

        # Replace the placeholder with the generated value
        fake_value = placeholder_to_generator[placeholder]()
        start, end = match.span()
        updated_text = updated_text[:start] + fake_value + updated_text[end:]

        # # Update the start and end positions for entities
        # print("Placeholder:", placeholder)
        # print("Start:", start)
        # print("End:", start + len(fake_value))
        entities.append((start, start + len(fake_value), placeholder))

    return updated_text, {"entities": entities}

In [ ]:
# # Template text
# template = "Hospital Name: Sunset General Hospital, ID - {PATIENT-ID}, Name - {PATIENT-NAME}, Address - {PATIENT-ADDRESS}, Type - {PATIENT-TYPE}, DOB - {PATIENT-DOB}, Gender - {PATIENT-GENDER}, Admit Date - {PATIENT-ADMIT-DATE}, Discharge Date - {PATIENT-DISCHARGE-DATE}, DocID - {PATIENT-DOCID}"

# updated_text, entities = replace_placeholders(template, placeholder_to_generator)
# print("Template:", template)
# print("Updated Text:", updated_text)
# print("Entities:", entities)

In [ ]:
text = "Hospital Name: Sunset General Hospital, ID - 48315993, Name - Gregory Henson, Address - 612 Cowan Rapids Suite 938 Dixonfort, GU 79621, Type - Oup, DOB - 2016-01-15, Gender - Other, Admit Date - 2021-04-13, Discharge Date - 1994-01-13, DocID - DOC-823-7359"
for i, char in enumerate(text):
    print(f"Index: {i}, Character: {char}")

METHOD TO GENERATE 'N' SAMPLES


In [20]:
def generate_training_data(placeholder_to_generator, templates, num_samples):
    training_data = []

    for _ in range(num_samples):
        template = random.choice(templates)
        updated_text, entities = replace_placeholders(
            template, placeholder_to_generator
        )
        training_data.append((updated_text, entities))

    return training_data

[('Hospital Name: Sunset General Hospital, ID - P-Zjs85063, Name - Jennifer Carpenter, Address - 50450 Kayla Road Apt. 667 Meredithchester, OR 60948, Type - Inp, DOB - 06/08/1975, Gender - M, Admit Date - 11/18/2021, Discharge Date - 04/03/2019, DocID - D-621-4676', {'entities': [(45, 55, 'PATIENT-ID'), (64, 82, 'PATIENT-NAME'), (94, 145, 'PATIENT-ADDRESS'), (154, 157, 'PATIENT-TYPE'), (165, 175, 'PATIENT-DOB'), (186, 187, 'PATIENT-GENDER'), (202, 212, 'PATIENT-ADMIT-DATE'), (231, 241, 'PATIENT-DISCHARGE-DATE'), (251, 261, 'PATIENT-DOCID')]}), ('Hospital Name: Sunrise Medical Center, Patient ID - TBx243682, Name - Sarah Mays', {'entities': [(52, 61, 'PATIENT-ID'), (70, 80, 'PATIENT-NAME')]}), ('Hospital Name: Morning General Hospital, ID - Wlj313558, Gender - M, Admit Date - 06/22/2022, Discharge Date - 10/05/2023, DocID - D-746-2440', {'entities': [(46, 55, 'PATIENT-ID'), (66, 67, 'PATIENT-GENDER'), (82, 92, 'PATIENT-ADMIT-DATE'), (111, 121, 'PATIENT-DISCHARGE-DATE'), (131, 141, 'PATI

In [43]:
# Example usage
num_samples = 1
training_data = generate_training_data(placeholder_to_generator, templates, num_samples)

for idx, sample in enumerate(training_data[1::-1]):
    print(f"Sample {idx}\nText:{sample[0]}\nEntities:{sample[1]['entities']}")

Sample 0
Text:Patient Registration Name: Darlene Clark ID: P-JhQ07967 Gender: Male Date of Birth: 12/19/1948 Address: 852 Carr Mount Suite 709 East Kelly, VI 65056 (Optional) Doctor: DOC-790-3706 (Optional)
Entities:[(27, 40, 'PATIENT-NAME'), (45, 55, 'PATIENT-ID'), (64, 68, 'PATIENT-GENDER'), (84, 94, 'PATIENT-DOB'), (104, 149, 'PATIENT-ADDRESS'), (169, 181, 'PATIENT-DOCID')]


In [23]:
text = "Hospital Name: Evening Medical Center, Patient ID - 80917920, Gender - Male, DOB - 12/08/1985, Admit Date - 02/07/2020, Discharge Date - 04/08/2019"
for i, char in enumerate(text):
    print(f"Index: {i}, Character: {char}")

Index: 0, Character: H
Index: 1, Character: o
Index: 2, Character: s
Index: 3, Character: p
Index: 4, Character: i
Index: 5, Character: t
Index: 6, Character: a
Index: 7, Character: l
Index: 8, Character:  
Index: 9, Character: N
Index: 10, Character: a
Index: 11, Character: m
Index: 12, Character: e
Index: 13, Character: :
Index: 14, Character:  
Index: 15, Character: E
Index: 16, Character: v
Index: 17, Character: e
Index: 18, Character: n
Index: 19, Character: i
Index: 20, Character: n
Index: 21, Character: g
Index: 22, Character:  
Index: 23, Character: M
Index: 24, Character: e
Index: 25, Character: d
Index: 26, Character: i
Index: 27, Character: c
Index: 28, Character: a
Index: 29, Character: l
Index: 30, Character:  
Index: 31, Character: C
Index: 32, Character: e
Index: 33, Character: n
Index: 34, Character: t
Index: 35, Character: e
Index: 36, Character: r
Index: 37, Character: ,
Index: 38, Character:  
Index: 39, Character: P
Index: 40, Character: a
Index: 41, Character: t
In

SAVING TO A PICKLE FILE


In [26]:
import os
import pickle

folder_path = "training_data"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

pickle_file_path = os.path.join(folder_path, "spacy_training_data.pkl")

with open(pickle_file_path, "wb") as file:
    pickle.dump(training_data, file)